# Lab 3: Scale with AgentCore Gateway & Shared Tools

In this lab, you'll centralize tools through **AgentCore Gateway** using the **MCP protocol**, enabling secure tool sharing across multiple agents.

## What you'll learn
- How to create an AgentCore Gateway with JWT authentication
- How to add Lambda-backed tools to the Gateway
- How to connect Claude Agent SDK to Gateway via `mcp_servers` config
- How to combine local MCP tools with remote Gateway tools

## Architecture

<div style="text-align:left">
    <img src="images/architecture_lab3_gateway.png" width="75%"/>
</div>

## Key Pattern: External MCP Server

Claude Agent SDK natively supports external MCP servers via URL:

```python
options = ClaudeAgentOptions(
    mcp_servers={
        "customer-support": sdk_server,          # In-process (local tools)
        "agentcore-gateway": {                    # External (Gateway tools)
            "type": "url",
            "url": gateway_url,
            "headers": {"Authorization": f"Bearer {jwt_token}"}
        }
    }
)
```

## Step 1: Setup

In [ ]:
import os
import json
import time

os.environ["CLAUDE_CODE_USE_BEDROCK"] = "1"
os.environ.pop("CLAUDECODE", None)

import boto3
from boto3.session import Session

boto_session = Session()
REGION = boto_session.region_name
account_id = boto3.client("sts").get_caller_identity()["Account"]

from utils.aws_helpers import (
    get_ssm_parameter,
    put_ssm_parameter,
    get_or_create_cognito_pool,
    load_api_spec,
)

print(f"Region: {REGION}")
print(f"Account: {account_id}")

## Step 2: Set Up Authentication (Cognito)

The Gateway uses JWT tokens for authentication. We create or retrieve a Cognito pool.

In [ ]:
# Get or create Cognito pool for JWT authentication
cognito_config = get_or_create_cognito_pool(refresh_token=True)

client_id = cognito_config["client_id"]
bearer_token = cognito_config["bearer_token"]
discovery_url = cognito_config["discovery_url"]

print(f"Client ID: {client_id}")
print(f"Discovery URL: {discovery_url}")
print(f"Bearer Token: {bearer_token[:20]}...")

## Step 3: Create AgentCore Gateway

The Gateway exposes tools via MCP protocol with JWT-based authorization.

In [ ]:
gateway_client = boto3.client("bedrock-agentcore-control", region_name=REGION)

# Get gateway role ARN from prerequisites
gateway_role_arn = get_ssm_parameter("/app/customersupport/agentcore/gateway_iam_role")

gateway_name = "customersupport-gw"

try:
    # Try to create a new gateway
    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn=gateway_role_arn,
        protocolType="MCP",
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration={
            "customJWTAuthorizer": {
                "allowedClients": [client_id],
                "discoveryUrl": discovery_url,
            }
        },
        description="Customer Support Gateway for shared tools",
    )
    gateway_id = create_response["gatewayId"]
    put_ssm_parameter("/app/customersupport/agentcore/gateway_id", gateway_id)

    # Wait for gateway to be ready
    print(f"Created gateway: {gateway_id}")
    print("Waiting for gateway to be ready...")
    while True:
        status_response = gateway_client.get_gateway(gatewayIdentifier=gateway_id)
        status = status_response["status"]
        if status == "READY":
            gateway_url = status_response["gatewayUrl"]
            break
        elif status in ["CREATE_FAILED", "FAILED"]:
            raise Exception(f"Gateway creation failed: {status}")
        time.sleep(10)
        print(f"  Status: {status}")

except Exception as e:
    # If gateway already exists, reuse it
    try:
        existing_gateway_id = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
        gateway_response = gateway_client.get_gateway(gatewayIdentifier=existing_gateway_id)
        gateway_id = existing_gateway_id
        gateway_url = gateway_response["gatewayUrl"]
        print(f"Using existing gateway: {gateway_id}")
    except Exception:
        raise Exception(f"Could not create or find gateway: {e}")

print(f"Gateway ID: {gateway_id}")
print(f"Gateway URL: {gateway_url}")

## Step 4: Add Lambda Target to Gateway

We add the Lambda function (check_warranty_status, web_search) as a Gateway target.

In [ ]:
# Get Lambda ARN from prerequisites
lambda_arn = get_ssm_parameter("/app/customersupport/agentcore/lambda_arn")

# Load API specification
api_spec = load_api_spec("prerequisite/lambda/api_spec.json")

# Create Lambda target
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_arn,
            "toolSchema": {"inlinePayload": api_spec},
        }
    }
}

try:
    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="LambdaTarget",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=[
            {"credentialProviderType": "GATEWAY_IAM_ROLE"}
        ],
    )
    target_id = create_target_response["targetId"]
    print(f"Created target: {target_id}")
    
    # Wait for target to be ready
    print("Waiting for target to be ready...")
    while True:
        target_status = gateway_client.get_gateway_target(
            gatewayIdentifier=gateway_id, targetId=target_id
        )
        status = target_status["status"]
        if status == "READY":
            break
        time.sleep(10)
        print(f"  Status: {status}")
    
    print(f"Target ready: {target_id}")
except Exception as e:
    if "already exists" in str(e).lower() or "conflict" in str(e).lower():
        print("Target already exists")
    else:
        raise

## Step 5: Connect Claude Agent SDK to Gateway

Now we configure the agent to use both local tools (via in-process MCP) and Gateway tools (via external MCP URL).

In [ ]:
from claude_agent_sdk import query, ClaudeAgentOptions
from agent import prompt_stream
from agent.mcp_server import get_mcp_server
from agent.gateway_client import get_gateway_mcp_config
from agent.prompts import SYSTEM_PROMPT

sdk_server = get_mcp_server()

# Get gateway config for MCP
gateway_mcp_config = get_gateway_mcp_config(bearer_token=bearer_token)


async def invoke_agent_with_gateway(user_input: str) -> str:
    """Invoke agent with both local and Gateway tools."""
    options = ClaudeAgentOptions(
        system_prompt=SYSTEM_PROMPT,
        mcp_servers={
            "customer-support": sdk_server,           # Local tools
            "agentcore-gateway": gateway_mcp_config,   # Gateway tools
        },
        allowed_tools=["mcp__customer-support__*", "mcp__agentcore-gateway__*"],
        permission_mode="bypassPermissions",
        max_turns=10,
    )

    response_text = ""
    async for msg in query(prompt=prompt_stream(user_input), options=options):
        if hasattr(msg, "result"):
            response_text = msg.result

    return response_text

print("Agent with Gateway configured!")
print(f"Local tools: get_return_policy, get_product_info, web_search, get_technical_support")
print(f"Gateway tools: check_warranty_status, web_search (Lambda-backed)")

## Step 6: Test Gateway Tools

In [ ]:
# Test: Warranty check (uses Gateway Lambda tool)
response = await invoke_agent_with_gateway(
    "Can you check the warranty status for serial number ABC12345? My email is customer@example.com"
)
print("=" * 60)
print("Query: Check warranty for serial number ABC12345")
print("=" * 60)
print(response)

In [ ]:
# Test: Combined local + gateway tools
response = await invoke_agent_with_gateway(
    "I want to know the return policy for laptops and also check my warranty for serial number XYZ98765"
)
print("=" * 60)
print("Query: Return policy + warranty check")
print("=" * 60)
print(response)

## Summary

In this lab, you connected Claude Agent SDK to AgentCore Gateway:

1. **Created Gateway** with JWT authentication (Cognito)
2. **Added Lambda target** with warranty check and web search tools
3. **Combined MCP servers** - local tools + Gateway tools in `ClaudeAgentOptions`
4. **Tested tool routing** - the agent automatically selects the right tool

### Key Architecture Difference
- **Strands**: Uses `MCPClient` wrapper with context manager
- **Claude Agent SDK**: Native `mcp_servers` config with URL-based external servers

In **Lab 4**, we'll deploy the agent to production using AgentCore Runtime.